## Utilities

In [3]:
from src import config

client_id, api_key = config.load_config()

print(f"client_id length: {len(client_id)}")
print(f"api_key length: {len(api_key)}")

client_id length: 5
api_key length: 32


In [5]:
from src.bungie_oauth import BungieAuth
import datetime

# perform oauth login to get the access token used in later requests.  It is good for 1 hour
print(
    "We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser."
)
access_token = BungieAuth(client_id).refresh_oauth_token()

# token is good for 1 hour, print out the time that it expires
expiration_time = datetime.datetime.now() + datetime.timedelta(hours=1)
print(
    f"Access token successfully acquired at: {datetime.datetime.now().isoformat()} and expires at: {expiration_time.isoformat()}"
)

We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser.
Please go to the following URL and authorize the app: https://www.bungie.net/en/oauth/authorize?client_id=22927&response_type=code&state=z4-2YjqB5ZVFIoW86fVZRPNFTHDNJdmzbcfhgWbdl3X-aONHFJcP9VvnXDjJwQ_CeHjY0azu35wsXZ0GvY_B3hZWgg-km6TyplqxHDV3LCwKjmOjXIaDifA5NtO_BdavBZG_PWZelqoxfaEKucZswGSruyoB0hyIclBZdjXQUs8=&redirect_uri=https://localhost:7777/
Stopping HTTPS server
Access token successfully acquired at: 2024-05-25T19:34:20.778640 and expires at: 2024-05-25T20:34:20.778473


In [6]:
# it's nice to have all of the manifest files downloaded locally into the data directory for reference
import os
import requests
from src.bungie_api import BungieApi

os.makedirs("data", exist_ok=True)

api = BungieApi(api_key, access_token)
manifest = api.get_manifest()

for key, value in manifest["Response"]["jsonWorldComponentContentPaths"]["en"].items():
    print(f"Downloading {key} from {value}")
    response = requests.get(
        f"https://www.bungie.net{value}",
        headers={
            "X-API-Key": api_key,
        },
    )
    with open(f"data/{key}.json", "w") as f:
        f.write(response.text)

### Ignore armor tagged as `infuse` or `junk` in DIM

If you [export your vault in DIM as a CSV](https://github.com/DestinyItemManager/DIM/wiki/Spreadsheets#), this can parse it and build a list of armor that the `d2armor` notebook should exclude from analysis. Adding "#ignore" to a DIM item's Notes field has the same effect — this can be used to exclude things like raid class items, which aren't junk, but are irrelevant to outfit calculations. 

In [2]:
%run src/process-dim-armor.py


Could not find /Users/tednaleid/Downloads/destinyArmor.csv
